# Data Cleaning

## Cleaning The Data

When dealing with numerical data, data cleaning often involves removing null values and duplicate data, dealing with outliers, etc. With text data, there are some common data cleaning techniques, which are also known as text pre-processing techniques.

With text data, this cleaning process can go on forever. There's always an exception to every cleaning step. So, we're going to follow the MVP (minimum viable product) approach - start simple and iterate. Here are a bunch of things you can do to clean your data. We're going to execute just the common cleaning steps here and the rest can be done at a later point to improve our results.

**Common data cleaning steps on all text:**
* Make text all lower case
* Remove punctuation
* Remove numerical values
* Remove common non-sensical text (/n)
* Tokenize text
* Remove stop words

**More data cleaning steps after tokenization:**
* Stemming / lemmatization
* Parts of speech tagging
* Create bi-grams or tri-grams
* Deal with typos
* And more...

In [14]:
import unicodedata
from nltk.stem import PorterStemmer, WordNetLemmatizer


In [15]:
# We can either keep it in dictionary format or put it into a pandas dataframe
import pandas as pd

data_df = pd.read_csv('Combined_Reddit_data.csv')

In [16]:
# Apply a first round of text cleaning techniques
import re
import string

def clean_text_round1(text):
    '''Make text lowercase, remove text in square brackets, remove punctuation and remove words containing numbers.'''
    text = text.lower()
    text = re.sub('\[.*?\]', '', text)
    text = re.sub('[%s]' % re.escape(string.punctuation), '', text)
    text = re.sub('\w*\d\w*', '', text)
    return text

round1 = lambda x: clean_text_round1(x)

In [17]:
data_df['Post'] = data_df['Post'].apply(round1)

In [18]:
# Apply a second round of cleaning
def clean_text_round2(text):
    '''Get rid of some additional punctuation and non-sensical text that was missed the first time around.'''
    text = re.sub('[‘’“”…]', '', text)
    text = re.sub('\n', '', text)
    # text = re.sub(r"(@\[A-Za-z0-9]+)|([^0-9A-Za-z \t])|(\w+:\/\/\S+)|^rt|http.+?", "", text)
    text = unicodedata.normalize('NFKD', text).encode('ascii','ignore').decode('utf-8')
    # text = unicodedata.normalize('NFKD', unicode(text, 'utf8'))

    return text

round2 = lambda x: clean_text_round2(x)

In [19]:
# Let's take a look at the updated text
data_df['Post'] = data_df['Post'].apply(round2)


In [20]:
# Stemming

porter_stemmer = PorterStemmer()
def stem_sentences(sentence):
    tokens = sentence.split()
    stemmed_tokens = [porter_stemmer.stem(token) for token in tokens]
    return ' '.join(stemmed_tokens)

data_df['Post'] = data_df['Post'].apply(stem_sentences)



In [21]:
# saving the cleaned dataframe into a csv

data_df.to_csv('Cleaned_Combined_Reddit_data.csv')

**NOTE:** This data cleaning aka text pre-processing step could go on for a while, but we are going to stop for now. After going through some analysis techniques, if you see that the results don't make sense or could be improved, you can come back and make more edits such as:
* Mark 'cheering' and 'cheer' as the same word (stemming / lemmatization)
* Combine 'thank you' into one term (bi-grams)
* And a lot more...

## Organizing The Data

I mentioned earlier that the output of this notebook will be clean, organized data in two standard text formats:
1. **Corpus - **a collection of text
2. **Document-Term Matrix - **word counts in matrix format

### Corpus

We already created a corpus in an earlier step. The definition of a corpus is a collection of texts, and they are all put together neatly in a pandas dataframe here.

### Document-Term Matrix

For many of the techniques we'll be using in future notebooks, the text must be tokenized, meaning broken down into smaller pieces. The most common tokenization technique is to break down text into words. We can do this using scikit-learn's CountVectorizer, where every row will represent a different document and every column will represent a different word.

In addition, with CountVectorizer, we can remove stop words. Stop words are common words that add no additional meaning to text such as 'a', 'the', etc.

In [22]:
# We are going to create a document-term matrix using CountVectorizer, and exclude common English stop words
from sklearn.feature_extraction.text import CountVectorizer

cv = CountVectorizer(stop_words='english')
data_cv = cv.fit_transform(data_df['Post'])
data_countVec = pd.DataFrame(data_cv.toarray(), columns=cv.get_feature_names_out())
data_countVec.index = data_df.index
data_countVec['Label'] = data_df['Label']
data_countVec

,ab,abnormal,absurdity,accelerate,accelerated,accelerating,access,according,accused,act,...,yes,york,youll,youre,ypocrite,ze,zealand,zero,zombie,Label
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,GlobalWarming
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,GlobalWarming
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,GlobalWarming
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,GlobalWarming
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,GlobalWarming
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
390,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,climate
391,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,climate
392,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,climate
393,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,climate


In [23]:
#  tempdf = data_countVec[data_countVec['Labels'] == 'climate_disruption']

In [24]:
#  tempdf =tempdf.sum(axis=0,numeric_only=True)

In [25]:
# TF-IDF

from sklearn.feature_extraction.text import TfidfVectorizer
tfidf = TfidfVectorizer(smooth_idf=True)
feature_matrix = tfidf.fit_transform(data_df['Post'])
# data_countVec = pd.DataFrame(data_cv.toarray(), columns=cv.get_feature_names_out())
data_tfidf = pd.DataFrame(data=feature_matrix.todense(), columns=tfidf.get_feature_names_out())
data_tfidf['Label'] = data_df['Label']
data_tfidf

,ab,abnormal,about,absurdity,accelerate,accelerated,accelerating,access,according,accused,...,youll,your,youre,yourself,ypocrite,ze,zealand,zero,zombie,Label
0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,GlobalWarming
1,0.0,0.0,0.151773,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,GlobalWarming
2,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,GlobalWarming
3,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,GlobalWarming
4,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,GlobalWarming
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
390,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,climate
391,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,climate
392,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,climate
393,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,climate


In [26]:
from sklearn.feature_extraction.text import TfidfTransformer 
tfIdfVectorizer=TfidfVectorizer(use_idf=True)
tfIdf = tfIdfVectorizer.fit_transform(data_df['Post'])
df = pd.DataFrame(tfIdf[0].T.todense(), index=tfIdfVectorizer.get_feature_names_out(), columns=["TF-IDF"])
df = df.sort_values('TF-IDF', ascending=False)
print (df.head(25))

               TF-IDF
shell        0.547976
bigger       0.292872
obscene      0.292872
windfall     0.292872
calls        0.292872
tax          0.273988
profit       0.260589
makes        0.260589
guardian     0.250197
after        0.234526
for          0.143896
the          0.086233
porsches     0.000000
preach       0.000000
pratley      0.000000
power        0.000000
potentially  0.000000
potential    0.000000
potent       0.000000
poor         0.000000
posts        0.000000
postmortem   0.000000
positive     0.000000
poorer       0.000000
posting      0.000000
